In [1]:
import pandas as pd
import datetime
import numpy as np
import os
import glob

print(datetime.datetime.now())
print(os.getcwd())

# 9/26/19 – 10/5/19
# 10/10 – 10/19

# vs the compariable days last year

2019-10-30 12:23:57.156910
/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Columbus_Day_Result


In [2]:
def recurisve_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [3]:
list_sales_file=list(recurisve_file_gen("/home/jian/BigLots/"))
list_sales_file=[x for x in list_sales_file if "daily" in x.lower()]
list_sales_file=[x for x in list_sales_file if x[-4:]==".txt"]
list_sales_file=[x for x in list_sales_file if "2019-" in x]
list_sales_file=[x for x in list_sales_file if x.split("MediaStorm_")[1][:10]>="2019-09-26"]
list_sales_file=[x for x in list_sales_file if x.split("MediaStorm_")[1][:10]<="2019-10-19"]
list_sales_file

['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-28/MediaStormDailySales20191001-111241-312.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-10-05/MediaStormDailySales20191008-120723-038.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-10-12/MediaStormDailySales20191015-111135-657.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-10-19/MediaStormDailySales20191022-111240-145.txt']

In [4]:
list_sales_file_2018=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
list_sales_file_2018=[x for x in list_sales_file_2018 if x.split("/MediaStormDailySalesHistory")[1][:8]>="20180925"]
list_sales_file_2018=[x for x in list_sales_file_2018 if x.split("/MediaStormDailySalesHistory")[1][:8]<="20181020"]
list_sales_file_2018

['/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180929.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181006.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181013.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181020.txt']

In [5]:
df_sales_2019=pd.DataFrame()
for file in list_sales_file:
    df=pd.read_table(file,dtype=str,sep="|")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"NonRewards","Rewards")
    #
    df_sales=df.groupby(["location_id","transaction_dt","rewards_label"])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales"})
    #
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','rewards_label']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(["location_id","transaction_dt","rewards_label"])['trans'].sum().to_frame().reset_index()
    
    df=pd.merge(df_sales,df_trans,on=["location_id","transaction_dt","rewards_label"],how="outer")
    df_sales_2019=df_sales_2019.append(df)
    print(datetime.datetime.now())
    

2019-10-30 12:24:51.387519
2019-10-30 12:25:41.396438
2019-10-30 12:26:20.617416
2019-10-30 12:27:11.941387


In [6]:
list_sales_file_2018

['/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180929.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181006.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181013.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181020.txt']

In [7]:
df_sales_2018=pd.DataFrame()
for file in list_sales_file_2018:
    df=pd.read_table(file,dtype=str,sep="|")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"NonRewards","Rewards")
    #
    df_sales=df.groupby(["location_id","transaction_dt","rewards_label"])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales"})
    #
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','rewards_label']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(["location_id","transaction_dt","rewards_label"])['trans'].sum().to_frame().reset_index()
    
    df=pd.merge(df_sales,df_trans,on=["location_id","transaction_dt","rewards_label"],how="outer")
    df_sales_2018=df_sales_2018.append(df)
    print(datetime.datetime.now())
    

2019-10-30 12:28:03.757670
2019-10-30 12:28:58.734015
2019-10-30 12:29:39.691701
2019-10-30 12:30:21.008618


In [8]:
list_test_date_2019=[datetime.date(2019,10,10)+datetime.timedelta(days=x) for x in range(10)]
list_control_date_2019=[datetime.date(2019,9,26)+datetime.timedelta(days=x) for x in range(10)]

list_test_date_2018=[x-datetime.timedelta(days=52*7) for x in list_test_date_2019]
list_control_date_2018=[x-datetime.timedelta(days=52*7) for x in list_control_date_2019]
#
df_test_19=pd.DataFrame({"transaction_dt":list_test_date_2019},index=['campaign']*len(list_test_date_2019)).reset_index()
df_test_19['year']=2019

df_control_19=pd.DataFrame({"transaction_dt":list_control_date_2019},index=['pre']*len(list_control_date_2019)).reset_index()
df_control_19['year']=2019

df_test_18=pd.DataFrame({"transaction_dt":list_test_date_2018},index=['campaign']*len(list_test_date_2018)).reset_index()
df_test_18['year']=2018


df_control_18=pd.DataFrame({"transaction_dt":list_control_date_2018},index=['pre']*len(list_control_date_2018)).reset_index()
df_control_18['year']=2018
#
df_date_type=pd.concat([df_test_19,df_control_19,df_test_18,df_control_18]).rename(columns={"index":"date_type"})
df_date_type['transaction_dt']=df_date_type['transaction_dt'].astype(str)

In [9]:
df_sales_2018=pd.merge(df_sales_2018,df_date_type,on="transaction_dt",how="left")
df_sales_2018=df_sales_2018[pd.notnull(df_sales_2018['date_type'])]

df_sales_2019=pd.merge(df_sales_2019,df_date_type,on="transaction_dt",how="left")
df_sales_2019=df_sales_2019[pd.notnull(df_sales_2019['date_type'])]

In [10]:
df_compariable=df_sales_2018.append(df_sales_2019)[['location_id','year','date_type','transaction_dt']].drop_duplicates()
df_compariable=df_compariable.groupby(['location_id','year','date_type'])['transaction_dt'].nunique().to_frame().reset_index()
df_compariable_overall=df_compariable.groupby(['location_id'])['transaction_dt'].sum().to_frame().reset_index()


In [11]:
excel_file=pd.ExcelFile("./BL_2019Q3_Labor_Day_IVL4_Test_SOTF_Stores_8.20.19_from_BL.xlsx")
excel_file.sheet_names

['Test', 'Control']

In [12]:
df_test=excel_file.parse("Test",dtype=str,skiprows=3,usecols=["Store#","SOTF ?"])
df_test=df_test[pd.notnull(df_test['Store#'])]
df_test=df_test.rename(columns={"SOTF ?":"SOTF","Store#":"location_id"})
df_test=df_test[['location_id','SOTF']]
df_test['Store_Type']="Test_Store"

In [13]:
df_control=excel_file.parse("Control",dtype=str,skiprows=2,usecols=["Store#","SOTF ?"])
df_control=df_control[pd.notnull(df_control['Store#'])]
df_control=df_control.rename(columns={"SOTF ?":"SOTF","Store#":"location_id"})
df_control=df_control[['location_id','SOTF']]
df_control['Store_Type']="Control_Store"

# note in the excel
df_control=df_control[df_control['location_id']!="1012"]

In [14]:
df_store_type=df_test.append(df_control)
df_store_type.shape,df_store_type['location_id'].nunique()

((190, 3), 190)

In [15]:
df_sales_both=df_sales_2019.append(df_sales_2018)
df_sales_both=pd.merge(df_sales_both,df_store_type,on="location_id",how="left")
df_sales_both['Store_Type']=np.where(pd.notnull(df_sales_both['Store_Type']),df_sales_both['Store_Type'],
                                    np.where(df_sales_both['location_id']=="6990","Online","Balance")
                                    )

In [16]:
df_sales_both['Store_Type'].unique()

array(['Balance', 'Control_Store', 'Test_Store', 'Online'], dtype=object)

In [17]:
writer=pd.ExcelWriter("./BL_ColumnsDay_Test_Performance_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_sales_both.to_excel(writer,"sales_data",index=False)
df_compariable_overall.to_excel(writer,"df_compariable_overall",index=False)
df_compariable.to_excel(writer,"df_compariable",index=False)
df_store_type.to_excel(writer,"df_store_type",index=False)
writer.save()